<a href="https://colab.research.google.com/github/rastri-dey/Data-Structures-and-Algorithms/blob/main/S3C_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Check Isomorphism

In [ ]:
import ctypes
import multiprocessing
from functools import lru_cache
from multiprocessing import Process

import rustworkx as rx

In [ ]:
def remove_ids(label):
    """
    Remove the id number from the label

    Assumes that if an id is present, it is of the form _number, e.g. car_2
    """
    if label is None:
        return None
    # hasattr: if an object(label) has a specific method or attribute
    # isinstance: if object is an instance of class or subclass
    if not (hasattr(label, 'name') and hasattr(label, 'label')):
        if isinstance(label, dict) and 'label' in label:
            return label['label']
        if hasattr(label, 'name'):
            under_index = label.name.rfind('_')
            if under_index == -1:
                return label.name
            else:
                return label.name[:under_index]
        return label
    under_index = label.name.rfind('_')
    return '%s:%s' % (label.label, label.name[:under_index] if under_index > 0 else label.name)
label = "car_2"
new_label = remove_ids(label)
print(new_label)  # Output: "car"


car_2


In [ ]:
@lru_cache
def get_hierarchy_check(force_consistent_ids=False):
    def hierarchy_check(label1, label2):
        # If they have the same label, they are equal
        if not force_consistent_ids:
            label1 = remove_ids(label1)
            label2 = remove_ids(label2)
        if label1 == label2:
            return True
    return hierarchy_check

In [ ]:
# comparing the structure of two ASGs without regard to the specific names of their nodes and edges
# In 2 graphs leaving node label ids or names, whether node & edge are same
def compare_asgs(asg1, asg2):
    return rx.digraph_is_isomorphic(asg1, asg2, id_order=False,
                                    node_matcher=get_hierarchy_check(),
                                    edge_matcher=get_hierarchy_check())

In [ ]:
def dict_to_string(d: dict):
    return str({key: d[key] for key in sorted(d.keys())})

In [ ]:
def get_class_counts(asg):
    """
    Returns a tuple of dictionaries (node_class_counts, edge_class_counts) with the class counts.

    If two graphs are isomorphic, then they must have the same number of every node and edge label (here called "class")
    This is used as a lightweight check as part of maybe_isomorphic to avoid running the more expensive algorithm.
    """
    node_class_counts = {}
    for node_index in asg.node_indices():
        label = remove_ids(asg[node_index])
        if label not in node_class_counts:
            node_class_counts[label] = 0
        node_class_counts[label] += 1
    edge_class_counts = {}
    for edge_index, (node1, node2, edge_data) in asg.edge_index_map().items():
        label = remove_ids(edge_data)
        if label not in edge_class_counts:
            edge_class_counts[label] = 0
        edge_class_counts[label] += 1
    return dict_to_string(node_class_counts), dict_to_string(edge_class_counts)

In [ ]:
def maybe_isomorphic(asg1, asg2):
    """Check metrics which are necessary but not sufficient for isomorphism and cheap to compute"""
    # if the graphs don't have the same size then they can't be isomorphic
    if asg1.num_nodes() != asg2.num_nodes() or asg1.num_edges() != asg2.num_edges():
        return False
    else:
        # if they don't have the same number of nodes and edges with equivalent labels, they can't be isomorphic
        asg1_class_counts = get_class_counts(asg1)
        asg2_class_counts = get_class_counts(asg2)
        if asg1_class_counts != asg2_class_counts:
            return False
    return True

In [ ]:
def __is_isomorphic(asg1, asg2, value: multiprocessing.Value = None):
    """Checks isomorphism directly. For improved performance, check maybe_isomorphic first"""
    result = compare_asgs(asg1, asg2)
    if value is not None:
        value.value = 1 if result else -1
    return result

NameError: name 'multiprocessing' is not defined

In [ ]:
def is_isomorphic(asg1, asg2, timeout=-1, check_preconditions=True):
    """
     Strict comparison assuming perfect equality. Can be run with timeout.
     If check_preconditions, checks maybe_isomorphic first. That time is excluded from the timeout.

     If timeout <=0, run without time limit. If timeout>0, run for at mose timeout seconds.
     If no result is found within that time, None is returned.
    """
    if check_preconditions and not maybe_isomorphic(asg1, asg2):
        return False
    if timeout <= 0:
        is_iso = __is_isomorphic(asg1, asg2)
    else:
        return_value = multiprocessing.Value(ctypes.c_byte, 0)
        p = Process(target=__is_isomorphic, args=(asg1, asg2, return_value))
        p.start()
        p.join(timeout)
        if p.is_alive() or return_value.value == 0:
            p.terminate()
            p.join()
            is_iso = None
        else:
            is_iso = return_value.value > 0
    return is_iso

## Generate Cluster

In [ ]:
import logging      # Log messages
import os           # interacting with the operating system
import sys          # interacting with the interpreter

import argparse
from pathlib import Path

from pipeline.Dataloader.dataloader_factory import DataLoaderFactory, DATALOADERS
from utils.dataset import Dataset

ModuleNotFoundError: No module named 'pipeline'

In [ ]:
def parse_cluster_args(arg_string):
    parser = argparse.ArgumentParser(description="Arguments for cluster generation")
    parser.add_argument(
        "-dt",
        "--dataset_type",
        type=str,
        choices=[dataloader for dataloader in DATALOADERS],
        required=False,
        help="Dataset type"
    )
    parser.add_argument(
        "-dp",
        "--dataset_path",
        type=Path,
        required=False,
        help="Dataset folder path"
    )
    parser.add_argument(
        "-sp",
        "--sgs_path",
        type=Path,
        required=False,
        help="Path to save/load SGs"
    )
    parser.add_argument(
        "-dsf",
        "--dataset_file",
        type=Path,
        required=False,
        help="Location to store data set file. If file exists, process will not regenerate and will exit. If file does "
             "not exist, then dataset_type, dataset_path, and sgs_path must be set."
    )
    parser.add_argument(
        "-j",
        "--threads",
        type=int,
        required=False,
        default=1,
        help="Number of threads to use for various operations."
    )
    parser.add_argument(
        "-max_per_thread",
        "--max_per_thread",
        type=int,
        required=False,
        default=512,
        help="Maximum number of graphs per thread. Larger groups will be split across threads and then merged."
    )
    parser.add_argument(
        "-load_threaded",
        "--load_threaded",
        type=bool,
        required=False,
        default=False,
        help="Whether to load the SGs in a threaded manner."
    )
    parser.add_argument('-v', '--verbose', action='store_true')

    return parser.parse_args(arg_string)

In [ ]:
'''
Parses the command-line arguments using the parse_cluster_args function.
Checks if the dataset file exists.

If the dataset file exists, it loads the dataset and extracts the scene graphs.
extract SG from util/dataset.py
Generates clusters based on the specified parameters.
Generates clusters from util/dataset.py

Saves the cluster information to a file.
Optionally, generates visualizations of the clusters.
If the dataset file does not exist, it prints an error message.
'''
def generate_clusters(arg_string):
    args = parse_cluster_args(arg_string)
    if args.verbose:
        logging.basicConfig(level=logging.INFO)
    if args.dataset_file is not None and os.path.exists(args.dataset_file):
        # The below structure can be used to load the datasets created by this method.
        # Note that the dataset path and sgs path args only need to be provided if the full file paths are needed
        # Otherwise, the dataset contains sufficient information for most other analysis
        dataset = Dataset.load_from_file(args.dataset_file, args.dataset_path, args.sgs_path)
    else:
        logging.info('Could not find dataset file %s, generating dataset' % args.dataset_file)
        dataloader = DataLoaderFactory(args.dataset_type, args.dataset_path, sgs_path=args.sgs_path,
                                       loader_type='Paths', shuffle=False)
        dataset = Dataset(dataloader,
                          threads=args.threads,
                          max_per_thead=args.max_per_thread,
                          load_threaded=args.load_threaded)
        if args.dataset_file is not None:
            logging.info('Saving dataset to file')
            dataset.save_to_file(args.dataset_file, args.dataset_path, args.sgs_path)


## Dataset: Set of clustered ASGs

In [3]:
import time # Provides various time-related functionalities
from pathlib import Path, PosixPath # Offers object-oriented filesystem paths
from utils.asg_compare import is_isomorphic, get_class_counts # Provides functions for comparing abstract syntax graphs (ASGs)
import os # Allows interaction with the operating system
import pickle # Used for object serialization and deserialization
from rustworkx import networkx_converter # Facilitates conversion between NetworkX and RustGraph objects
from functools import lru_cache # Provides higher-order functions and decorators
import functools
import json # Enables working with JSON data
import multiprocessing # Supports parallel processing using multiple processors
from multiprocessing import Pool # Represents a pool of worker processes for parallel execution
from concurrent.futures import ProcessPoolExecutor as ProcessPoolExecutor
from typing import Tuple, List, Dict

from tqdm import tqdm
import logging
from pipeline.Dataloader.abstract_dataloader import AbstractDataloader

import rustworkx as rx
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

ModuleNotFoundError: No module named 'utils'

In [1]:
class Node:
    def __init__(self, name, base_class=None):
        self.name = name
        self.base_class = base_class

    def __repr__(self):
        return str(self.name)

class SGUnickler(pickle.Unpickler):
    def find_class(self, module, name):
        # print(module, name)
        if (module == '__main__' or module == 'roadscene2vec.scene_graph.nodes') and name == 'Node':
            return Node
        return super().find_class(module, name)

In [2]:
# loads a scene graph from a file & converts it to RustGraph or NetworkX object
# sg_file: The path to the file containing the pickled scene graph
# convert_to_rustworkx: A boolean flag indicating whether to convert the loaded scene graph to a RustGraph object

@lru_cache(maxsize=int(os.getenv('SG_CACHE_SIZE', default='128')))
def load_sg(sg_file, convert_to_rustworkx=True):
    with open(sg_file, 'rb') as f:
        # sg = pickle.load(f)
        sg = SGUnickler(f).load()
    # convert from networkx to rustworkx for efficiency
    if convert_to_rustworkx:
        sg = networkx_converter(sg)
    return sg

NameError: name 'lru_cache' is not defined

In [1]:
# checks if loaded scene graph is same as an empty scene graph
def is_empty_sg(sg) -> bool:
    if isinstance(sg, str) or isinstance(sg, PosixPath):
        sg = load_sg(sg)
    return is_isomorphic(sg, EMPTY_SG)

In [ ]:
# retrieves metadata about a scene graph, including
# the number of nodes, number of edges, and class counts for the nodes

def get_sg_metadata(sg, bypass_cache=False) -> Tuple[int, int]:
    if isinstance(sg, str) or isinstance(sg, PosixPath):
        if bypass_cache:
            sg = load_sg.__wrapped__(sg)
        else:
            sg = load_sg(sg)
    return sg.num_nodes(), sg.num_edges(), get_class_counts(sg)

In [ ]:
# searches for a scene graph isomorphic to the one provided in sg1_file
# within a list of other scene graphs specified by sg_files

def find_iso(sg1_file, cluster_keys, sg_files,
             check_preconditions: bool = True, timeout: float = -1, verbose: bool = False, leave: bool = True):
    """Find the cluster that the SG given by sg1_file belongs in, i.e. is isomorphic to"""
    sg1 = load_sg(sg1_file)
    with tqdm(total=len(cluster_keys), disable=not verbose, leave=leave) as pbar:
        for key2 in cluster_keys:
            sg2 = load_sg(sg_files[key2])
            if is_isomorphic(sg1, sg2, timeout=timeout, check_preconditions=check_preconditions):
                pbar.update(len(cluster_keys))
                # return the key as soon as we find it
                return key2
            pbar.update(1)
    # if we have not found a key after looking through all of them, return None
    return None

In [4]:
# Dummy
from tqdm import tqdm
with tqdm(total=100) as pbar:
    for i in range(100):
        # Do something
        pbar.update(1)

100%|██████████| 100/100 [00:00<00:00, 370849.16it/s]


In [5]:
# There is a file named empty_sg.pkl in the parent directory
# of the current Python file
EMPTY_SG = load_sg(Path(__file__).parent / 'empty_sg.pkl')
EMPTY_SG_METADATA = get_sg_metadata(EMPTY_SG)

NameError: name 'load_sg' is not defined

In [ ]:
class IsoTimeoutError(Exception):
    def __init__(self, file1, file2):
        self.file1 = file1
        self.file2 = file2

    def __str__(self):
        return f'"{self.file1}", "{self.file2}"'

    def get_files(self):
        return self.file1, self.file2